In [1]:
from predictors import XGBoostClassifier, LightGBMClassifier, WeightedEnsembleClassifier
from custom_voter_boost_all_data import (
    all_estimators,
    compute_metrics,
    lgbm_classifiers,
)


[0]	train-aucpr:0.76053	validation-aucpr:0.45490
[9]	train-aucpr:0.93437	validation-aucpr:0.67814
[0]	train-aucpr:0.71779	validation-aucpr:0.44752
[9]	train-aucpr:0.91065	validation-aucpr:0.65301
[0]	train-aucpr:0.74005	validation-aucpr:0.38708
[9]	train-aucpr:0.94620	validation-aucpr:0.66106
[0]	train-aucpr:0.61725	validation-aucpr:0.46993
[10]	train-aucpr:0.76580	validation-aucpr:0.62708
[0]	train-aucpr:0.59382	validation-aucpr:0.47189
[9]	train-aucpr:0.77973	validation-aucpr:0.62214
[0]	train-aucpr:0.62183	validation-aucpr:0.46929
[9]	train-aucpr:0.80654	validation-aucpr:0.64429
[LightGBM] [Info] Number of positive: 1098, number of negative: 21689
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004635 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16320
[LightGBM] [Info] Number of data points in the train set: 22787, number of used features: 64
[LightGBM] [Info] [binary:BoostFromScore]: pavg=

In [2]:
voter = WeightedEnsembleClassifier(estimators=lgbm_classifiers, weights=None)


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

data = pd.read_csv("./clean_data.csv")
targets = data.pop(list(data.columns)[-1])
X_train, X_test, y_train, y_test = train_test_split(
    data, targets, test_size=0.3, random_state=121, stratify=targets
)
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.67, random_state=121, stratify=y_test
)
data = {}
data["train"] = [X_train, y_train]
data["val"] = [X_val, y_val]
data["test"] = [X_test, y_test]

unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))
class_weights = {}
total_samples = np.sum(counts)
for cls in class_counts:
    class_weights[cls] = total_samples / (len(class_counts) * class_counts[cls])
sample_weights = np.array([class_weights[cls] for cls in y_train])


In [4]:
wts, mean_threshold = voter.score(data["val"][0], data["val"][1])


In [5]:
mean_threshold

0.7703772888406947

In [6]:
voter_2 = WeightedEnsembleClassifier(estimators=lgbm_classifiers, weights=wts)
preds = voter_2.predict(X=data["test"][0], threshold=mean_threshold)


In [7]:
from pprint import pprint

pprint(compute_metrics(y_pred=preds, y_test=data["test"][1]))


{'f1_score': 0.7072243346007604,
 'precision': 0.8815165876777251,
 'recall': 0.5904761904761905}
